<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Parking_spot_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import os
import cv2
import shutil
import pickle
import skimage
import sklearn
import numpy as np
import pandas as pd

#Image Classifier

We buid a modelto predict the parking spots, if they are empty or not.

[Data](https://drive.google.com/file/d/11DyZ165lZGzULEZSQNofyy9A8xaYgFJ2)

In [2]:
zip_folder = '/content/drive/MyDrive/DATA/car-parking-slot-empty-notempty.zip'

In [3]:
shutil.copyfile(zip_folder, '/content/data.zip')
!unzip \*.zip && rm *.zip

'/content/data.zip'

In [9]:
data = []
labels = []
for id, cat in enumerate(['empty', 'not_empty']):
  path = f'./clf-data/{cat}/'
  for img in os.listdir(path):
    img = skimage.io.imread(path+img)
    img = skimage.transform.resize(img, (15, 15))
    data.append(img.flatten())
    labels.append(id)

data = np.asarray(data)
labels = np.asarray(labels)

In [13]:
data.shape,labels.shape

((6090, 675), (6090,))

In [19]:
train_x, test_x, train_y, test_y = sklearn.model_selection.train_test_split(data, labels, test_size=0.1, random_state=32)

In [20]:
train_x.shape, test_x.shape,

((5481, 675), (609, 675))

In [26]:
clf = sklearn.svm.SVC()

parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

grid_search = sklearn.model_selection.GridSearchCV(clf, parameters)

grid_search.fit(train_x, train_y)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}])

In [28]:
best_estimator = grid_search.best_estimator_

pred = best_estimator.predict(test_x)

score = sklearn.metrics.accuracy_score(pred, test_y)

print('{}% of samples were correctly classified'.format(str(score * 100)))

100.0% of samples were correctly classified


In [30]:
pickle.dump(best_estimator, open('./model.p', 'wb'))

In [31]:
shutil.copyfile('./model.p', './drive/MyDrive/DATA/model.p')

'./drive/MyDrive/DATA/model.p'

#Parking Spot Detector

In [37]:
MODEL = pickle.load(open("./drive/MyDrive/DATA/model.p", "rb"))


mask_img = cv2.imread('./drive/MyDrive/DATA/mask_1920_1080.png')
mask_crop_img = cv2.imread('./drive/MyDrive/DATA/mask_crop.png')

In [ ]:
def empty_or_not(spot_bgr):

    flat_data = []

    img_resized = skimage.transform.resize(spot_bgr, (15, 15, 3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)

    y_output = MODEL.predict(flat_data)

    if y_output == 0:
        return EMPTY
    else:
        return NOT_EMPTY


def get_parking_spots_bboxes(connected_components):
    (totalLabels, label_ids, values, centroid) = connected_components

    slots = []
    coef = 1
    for i in range(1, totalLabels):

        # Now extract the coordinate points
        x1 = int(values[i, cv2.CC_STAT_LEFT] * coef)
        y1 = int(values[i, cv2.CC_STAT_TOP] * coef)
        w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
        h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)

        slots.append([x1, y1, w, h])

    return slots